
# 📘 Caderno de Exercícios – Probabilidade em Python para IA (com soluções)
Autor: ChatGPT — GPT-5 Thinking  
Descrição: Notebook com **soluções prontas** para 10 exercícios práticos de probabilidade aplicada à IA, incluindo simulações, distribuições e Naive Bayes.

> Observação: Execute as células na ordem. Gráficos usam apenas `matplotlib`.


In [1]:

# Imports principais (execute primeiro)
import random
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

# Bibliotecas de ciência de dados/IA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc

# Bibliotecas científicas
from scipy import stats
from scipy import integrate

# Reprodutibilidade
random.seed(42)
np.random.seed(42)

print("Pacotes carregados com sucesso.")


Pacotes carregados com sucesso.



## Exercício 1 – Sorteio em urna (teórico vs simulação)

**Enunciado**: Uma urna contém 3 bolas vermelhas e 2 bolas azuis.  
1) Probabilidade teórica de sortear uma bola vermelha.  
2) Simule 10.000 sorteios e estime a probabilidade experimental.  
3) Compare os resultados.


In [4]:

# Solução
urna = ["vermelha"] * 3 + ["azul"] * 2

# 1) Teórica
p_teorica = 3 / 5

# 2) Simulação
n = 10_000
sorteios = [random.choice(urna) for _ in range(n)]
p_exp = sorteios.count("vermelha") / n

print(f"Probabilidade teórica (vermelha): {p_teorica:.4f}")
print(f"Probabilidade experimental (vermelha): {p_exp:.4f}")
print(f"Erro absoluto: {abs(p_teorica - p_exp):.4f}")


Probabilidade teórica (vermelha): 0.6000
Probabilidade experimental (vermelha): 0.5982
Erro absoluto: 0.0018



## Exercício 2 – Lançamento de dado (par)

**Enunciado**: Um dado justo de 6 lados é lançado uma vez.  
1) Probabilidade de sair número **par**.  
2) Simule 100.000 lançamentos e estime a probabilidade experimental.


In [ ]:

# Solução
lados = [1,2,3,4,5,6]
p_teorica = 3 / 6  # {2,4,6}

N = 100_000
rolagens = np.random.choice(lados, size=N, replace=True)
p_exp = np.isin(rolagens, [2,4,6]).mean()

print(f"Probabilidade teórica (par): {p_teorica:.4f}")
print(f"Probabilidade experimental (par): {p_exp:.4f}")
print(f"Erro absoluto: {abs(p_teorica - p_exp):.4f}")



## Exercício 3 – Probabilidade condicional com baralho

**Enunciado**: Num baralho de 52 cartas, calcule a probabilidade de tirar uma carta **vermelha** dado que é uma **figura** (J, Q, K).  
1) Resolva teoricamente.  
2) Valide via simulação.


In [ ]:

# Solução
naipes_vermelhos = {"♥", "♦"}
naipes_pretos = {"♣", "♠"}
ranks = ['A'] + [str(i) for i in range(2,11)] + ['J','Q','K']
naipes = list(naipes_vermelhos | naipes_pretos)
baralho = [(r, n) for r in ranks for n in naipes]

# Teórica: figuras são J,Q,K (3 por naipe, 12 no total). Metade é vermelha (6).
p_teorica = 6 / 12

# Simulação: condicionar ao evento "é figura"
N = 200_000
amostras = np.random.choice(len(baralho), size=N, replace=True)
figura = [(baralho[i][0] in {'J','Q','K'}) for i in amostras]
vermelha = [(baralho[i][1] in naipes_vermelhos) for i in amostras]

# P(vermelha | figura) = P(vermelha e figura)/P(figura)
fig_e_verm = np.mean([f and v for f, v in zip(figura, vermelha)])
p_fig = np.mean(figura)
p_exp = fig_e_verm / p_fig

print(f"P(vermelha | figura) teórica: {p_teorica:.4f}")
print(f"P(vermelha | figura) experimental: {p_exp:.4f}")
print(f"Erro absoluto: {abs(p_teorica - p_exp):.4f}")



## Exercício 4 – Moeda viciada (Distribuição Binomial)

**Enunciado**: Moeda com P(cara)=0,6.  
1) Simule 100 lançamentos e conte caras.  
2) Repita 1000 vezes.  
3) Plote um histograma.


In [ ]:

# Solução
n = 100      # lançamentos por experimento
p = 0.6      # prob de cara
experimentos = 1000

resultados = np.random.binomial(n, p, size=experimentos)

print(f"Média teórica: {n*p:.2f} | Desvio teórico: {math.sqrt(n*p*(1-p)):.2f}")
print(f"Média simulada: {resultados.mean():.2f} | Desvio simulado: {resultados.std(ddof=1):.2f}")

plt.figure()
plt.hist(resultados, bins=20, density=True)
plt.title("Binomial: nº de caras em 100 lançamentos (1000 repetições)")
plt.xlabel("Nº de caras")
plt.ylabel("Densidade")
plt.show()



## Exercício 5 – Tempo até o primeiro sucesso (Geométrica)

**Enunciado**: Uma lâmpada tem 5% de chance de queimar a cada hora (independente).  
1) Modele com distribuição **geométrica**.  
2) Simule 10.000 lâmpadas.  
3) Calcule a média esperada.


In [ ]:

# Solução
p = 0.05  # probabilidade de "queimar" por hora
amostras = np.random.geometric(p, size=10_000)  # nº de horas até a 1ª queima (1-indexed)

media_teorica = 1 / p
media_empirica = amostras.mean()

print(f"Média teórica (horas): {media_teorica:.2f}")
print(f"Média empírica (horas): {media_empirica:.2f}")

plt.figure()
plt.hist(amostras, bins=50, density=True)
plt.title("Distribuição Geométrica (p=0.05) – horas até a 1ª queima")
plt.xlabel("Horas")
plt.ylabel("Densidade")
plt.show()



## Exercício 6 – Alturas (Normal)

**Enunciado**: Alturas ~ Normal(μ=1.70, σ=0.10).  
1) Gere 10.000 amostras.  
2) Plote histograma.  
3) Probabilidade de altura > 1.80 m?


In [ ]:

# Solução
mu, sigma = 1.70, 0.10
amostras = np.random.normal(mu, sigma, size=10_000)

# Probabilidade teórica: 1 - CDF_normal(1.80)
p_maior_180 = 1 - stats.norm.cdf(1.80, loc=mu, scale=sigma)
print(f"P(altura > 1.80) teórica: {p_maior_180:.4f}")

plt.figure()
plt.hist(amostras, bins=30, density=True)
plt.title("Alturas ~ Normal(μ=1.70, σ=0.10)")
plt.xlabel("Altura (m)")
plt.ylabel("Densidade")
plt.show()



## Exercício 7 – Classificação com Naive Bayes (texto)

**Enunciado**: Construa um classificador Multinomial Naive Bayes para os emails e classifique `"trabalho amanhã com promoção"`.


In [ ]:

# Solução
emails = [
    "ganhe dinheiro rápido",
    "promoção imperdível",
    "encontro amanhã no trabalho",
    "relatório do projeto",
    "desconto exclusivo para você"
]
rotulos = ["spam", "spam", "não spam", "não spam", "spam"]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(emails)

modelo = MultinomialNB()
modelo.fit(X, rotulos)

novo_email = ["trabalho amanhã com promoção"]
X_novo = vectorizer.transform(novo_email)
pred = modelo.predict(X_novo)[0]
probas = modelo.predict_proba(X_novo)[0]

print("Vocabulário:", sorted(vectorizer.vocabulary_.keys()))
print("Classificação:", pred)
print("Probabilidades por classe (na ordem modelo.classes_):")
print(modelo.classes_)
print(probas)



## Exercício 8 – Diagnóstico médico (Bayes)

**Enunciado**: Prevalência 1%. Sensibilidade 95%. Especificidade 90%.  
1) Calcule P(doença | teste positivo).  
2) Simule 100.000 pessoas e valide.


In [ ]:

# Solução

preval = 0.01
sens = 0.95
espec = 0.90

# Fórmula de Bayes:
# P(D|+) = (sens * preval) / (sens*preval + (1-espec)*(1-preval))
p_dado_pos = (sens * preval) / (sens*preval + (1 - espec) * (1 - preval))
print(f"P(doença | positivo) (Bayes): {p_dado_pos:.4f}")

# Simulação
N = 100_000
tem_doenca = np.random.rand(N) < preval
teste_positivo = np.empty(N, dtype=bool)
# Quem tem doença: positivo com prob sens
teste_positivo[tem_doenca] = np.random.rand(tem_doenca.sum()) < sens
# Quem não tem: falso positivo com prob (1 - espec)
teste_positivo[~tem_doenca] = np.random.rand((~tem_doenca).sum()) < (1 - espec)

p_empirica = tem_doenca[teste_positivo].mean()
print(f"P(doença | positivo) (simulação): {p_empirica:.4f}")
print(f"Erro absoluto: {abs(p_empirica - p_dado_pos):.4f}")



## Exercício 9 – Monte Carlo da área sob a sigmoide

**Enunciado**: Estime a integral de `sigm(x)=1/(1+e^{-x})` em [-6, 6] por Monte Carlo e compare com valor numérico.


In [ ]:

# Solução
def sigm(x):
    return 1.0 / (1.0 + np.exp(-x))

a, b = -6, 6
N = 200_000
xs = np.random.uniform(a, b, size=N)
mc_est = (b - a) * np.mean(sigm(xs))

# Valor "real" numérico via quadratura
real_val, err_est = integrate.quad(lambda x: 1.0 / (1.0 + np.exp(-x)), a, b)

print(f"Monte Carlo: {mc_est:.6f}")
print(f"Quadratura (scipy): {real_val:.6f} (erro estimado: {err_est:.2e})")
print(f"Erro absoluto: {abs(mc_est - real_val):.6f}")

# Visualização da função (opcional)
grid = np.linspace(a, b, 400)
plt.figure()
plt.plot(grid, sigm(grid))
plt.title("Função Sigmoide em [-6, 6]")
plt.xlabel("x")
plt.ylabel("sigm(x)")
plt.show()



## Exercício 10 – Projeto: Classificação com Naive Bayes + ROC

**Enunciado**:  
1) Gere dataset sintético.  
2) Treine **GaussianNB**.  
3) Probabilidades previstas.  
4) Plote curva ROC.


In [ ]:

# Solução
X, y = make_classification(
    n_samples=2000, n_features=8, n_informative=5, n_redundant=1,
    n_classes=2, class_sep=1.2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

gnb = GaussianNB()
gnb.fit(X_train, y_train)

probas = gnb.predict_proba(X_test)[:, 1]
fpr, tpr, thr = roc_curve(y_test, probas)
roc_auc = auc(fpr, tpr)

print(f"AUC: {roc_auc:.4f}")
print("Exemplo de probabilidades previstas (10 primeiras):")
print(probas[:10])

plt.figure()
plt.plot(fpr, tpr, label=f"ROC (AUC = {roc_auc:.3f})")
plt.plot([0,1], [0,1], linestyle='--')
plt.title("Curva ROC – GaussianNB")
plt.xlabel("Falso positivo (FPR)")
plt.ylabel("Verdadeiro positivo (TPR)")
plt.legend()
plt.show()
